# 보팅 (xgboost + lgbm + rf)

In [4]:
import pandas as pd
import os
import matplotlib as mpl
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
print(mpl.__version__)
# seaborn 스타일 사용
plt.style.use('seaborn-notebook')

# matplotlib 그래프 한글폰트 깨질 때 대처(Mac & Window)
import platform
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# Mac 인 경우
    rc('font', family='AppleGothic')
    
# 그래프에서 마이너스 기호가 표시되도록 하는 설정
mpl.rcParams['axes.unicode_minus'] = False

import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline


3.4.3


In [5]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()

VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(train_X,train_y)

In [ ]:
pred = VC.predict(test)

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['target'] = pred

submission

submission.to_csv('submission.csv',index=False)

# pycaret

In [ ]:
from pycaret.classification import *

In [ ]:
setup_clf = setup(data = train, target = 'target', fold_shuffle=True, use_gpu=True, silent=True)

In [ ]:
top10 = compare_models(sort='Accuracy', n_select=10, fold = 5)

In [ ]:
tuned_top10 = [tune_model(i, fold = 5) for i in top10]

In [ ]:
blender_top10 = blend_models(estimator_list=tuned_top10)

In [ ]:
final_model = finalize_model(blender_top10)
prediction = predict_model(final_model, data= test)

In [ ]:
prediction

In [ ]:
prediction['Label']

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['target'] = prediction['Label']

submission

submission.to_csv('submission.csv',index=False)

In [ ]:
# extratree => 0.87
# xgboost => 0.87

# 보팅 (extratree + LGBM + catboost)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import catboost as ctb

In [ ]:
LGBM = LGBMClassifier(max_bin=300, num_leaves=24)
xtree = ExtraTreesClassifier(max_depth=6)
ctb = ctb.CatBoostClassifier(silent=True, depth=6, l2_leaf_reg=7, learning_rate=0.1)

VC = VotingClassifier(estimators=[('xt',xtree),('ctb',ctb),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(train_X, train_y)

In [ ]:
pred = VC.predict(test)

In [ ]:
submission = pd.read_csv('sample_submission.csv')

submission['target'] = pred

submission

submission.to_csv('submission.csv',index=False)

# 정확도 계산 - test셋을 분리 후 정확도 보기

## 보팅으로 정확도 보기

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y,
                                                   test_size = 0.2, random_state=1)

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
RF = RandomForestClassifier()

VC = VotingClassifier(estimators=[('rf',RF),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train, y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.8373

## ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
xtree = ExtraTreesClassifier()
xtree.fit(X_train, y_train)

In [ ]:
pred = xtree.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.8030

## 보팅으로 정확도 보기2 (RF -> extratree)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y,
                                                   test_size = 0.2, random_state=1)

In [ ]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
LGBM = LGBMClassifier()
XGB = XGBClassifier()
xtree = ExtraTreesClassifier()

VC = VotingClassifier(estimators=[('xt',xtree),('xgb',XGB),('lgbm',LGBM)],voting = 'soft')

In [ ]:
VC.fit(X_train, y_train)

In [ ]:
pred = VC.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.8415

## 랜덤포레스트로 정확도 보기

In [ ]:
rf_clf = RandomForestClassifier()
rf_clf.fit(X_train , y_train)
pred = rf_clf.predict(X_test)
pred

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.7709

## xgboost로 정확도 보기

In [ ]:
import xgboost as xgb 

In [ ]:
dtrain = xgb.DMatrix(data=X_train , label=y_train)
dtest = xgb.DMatrix(data=X_test , label=y_test)

In [ ]:
params = { 'max_depth':3,
           'eta': 0.1,   # 학습률
           'objective':'multi:softmax',
           'eval_metric':'merror',
           'early_stoppings':10
        }

# 학습 횟수
num_rounds = 100

In [ ]:
# 데이터를 학습용은 ‘train’ , 검증용은 ‘eval’로 표시
wlist = [(dtrain,'train'),(dtest,'eval') ]

In [ ]:
# 하이퍼 파라미터와 early stopping 파라미터를 train( ) 함수의 파라미터로 전달
xgb_model = xgb.train(params = params, dtrain=dtrain, num_boost_round=num_rounds, evals=wlist)  #xgb.fit이 아닌 xgb.train
xgb_model

## Catboost

In [ ]:
import catboost as ctb

In [ ]:
model_CBC = ctb.CatBoostClassifier()

In [ ]:
model_CBC.fit(X_train, y_train)

In [ ]:
pred = model_CBC.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))

#예측 정확도: 0.8458

## 보팅으로 정확도 보기2 (xgboost => catboost)

In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y,
                                                   test_size = 0.2, random_state=1)

In [22]:
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
import catboost as ctb

In [23]:
LGBM = LGBMClassifier(max_bin=100, num_leaves=15, max_depth=6)
xtree = ExtraTreesClassifier(max_depth=6)
ctb = ctb.CatBoostClassifier(silent=True, depth=6, l2_leaf_reg=7, learning_rate=0.1)


VC = VotingClassifier(estimators=[('xt',xtree),('ctb',ctb),('lgbm',LGBM)],voting = 'soft')

In [24]:
VC.fit(X_train, y_train)

VotingClassifier(estimators=[('xt', ExtraTreesClassifier(max_depth=6)),
                             ('ctb',
                              <catboost.core.CatBoostClassifier object at 0x000002105381F160>),
                             ('lgbm',
                              LGBMClassifier(max_bin=100, max_depth=6,
                                             num_leaves=15))],
                 voting='soft')

In [25]:
pred = VC.predict(X_test)

In [26]:
pred

array([0, 1, 0, 3, 0, 3, 1, 3, 0, 2, 2, 3, 0, 1, 0, 3, 1, 2, 1, 0, 2, 2,
       0, 0, 1, 2, 3, 3, 2, 1, 0, 0, 3, 2, 3, 2, 0, 1, 3, 3, 1, 0, 2, 3,
       3, 3, 0, 1, 2, 3, 0, 2, 1, 1, 3, 2, 1, 3, 0, 2, 2, 3, 1, 1, 2, 3,
       1, 0, 2, 3, 1, 3, 1, 0, 2, 1, 0, 0, 2, 1, 2, 3, 1, 3, 1, 1, 3, 1,
       1, 3, 1, 2, 3, 0, 1, 1, 0, 2, 3, 1, 2, 2, 3, 0, 3, 3, 2, 2, 1, 1,
       2, 2, 0, 2, 1, 0, 2, 0, 2, 3, 3, 3, 1, 0, 0, 0, 1, 3, 2, 2, 2, 3,
       0, 2, 3, 0, 3, 1, 2, 1, 1, 1, 2, 0, 2, 3, 1, 2, 0, 1, 1, 0, 2, 2,
       0, 1, 0, 0, 1, 3, 0, 1, 1, 1, 2, 3, 2, 0, 1, 2, 2, 0, 3, 3, 3, 1,
       2, 3, 2, 1, 1, 0, 0, 2, 2, 3, 2, 2, 1, 0, 1, 3, 1, 1, 1, 3, 1, 1,
       2, 2, 3, 0, 0, 0, 1, 2, 3, 3, 0, 2, 1, 3, 0, 2, 3, 2, 2, 1, 3, 0,
       3, 1, 3, 1, 0, 3, 0, 3, 1, 2, 3, 1, 2, 1, 1, 1, 3, 2, 2, 1, 3, 3,
       1, 3, 2, 1, 3, 3, 1, 2, 3, 0, 2, 0, 2, 2, 1, 1, 1, 1, 3, 0, 2, 2,
       0, 1, 0, 3, 1, 1, 1, 2, 0, 0, 2, 1, 3, 3, 0, 2, 3, 2, 0, 0, 2, 0,
       1, 1, 3, 2, 2, 1, 2, 1, 3, 2, 2, 1, 1, 1, 1,

In [27]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, pred)))


# 예측 정확도: 0.8672

예측 정확도: 0.8608


# 딥러닝 기반

In [ ]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint,EarlyStopping
import tensorflow as tf

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

train_X = train.drop('target', axis=1)
train_y = train.target

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y,
                                                   test_size = 0.2, random_state=1)

In [ ]:
model = Sequential()
model.add(Dense(40, input_dim=32, activation = 'relu'))
model.add(Dense(40, activation='relu'))
#model.add(Dense(50, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
model.fit(train_X, train_y, epochs=30, batch_size=5)

In [ ]:
# 예측 값과 실제 값의 비교
Y_prediction = model.predict(test)
# flatten : 데이터 배열이 몇 차원이든 모두 1차원으로 바꿔 읽기 쉽게 해 주는 함수

In [ ]:
value_list=[]
for i in range(len(Y_prediction)):
    maxindex = np.argmax(Y_prediction[i])
    value_list.append(maxindex)

In [ ]:
from sklearn.metrics import accuracy_score
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test, value_list)))


# 예측 정확도: 0.6146

In [ ]:
submission = pd.read_csv('submission.csv')

submission['target2'] = value_list

submission

submission.to_csv('submission.csv',index=False)

In [ ]:
submission.target2.value_counts()

In [ ]:
submission.target.value_counts()

## 파라미터 튜닝

In [ ]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [ ]:
model_CAT = ctb.CatBoostClassifier(silent = True)

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_cv_CAT = GridSearchCV(model_CAT, param_grid=grid, cv=3 , n_jobs=-1)
grid_cv_CAT.fit(X_train, y_train)
print('최적 하이퍼 파라미터: ', grid_cv_CAT.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_CAT.best_score_))

In [ ]:
model_XGB = XGBClassifier(eval_metric='logloss', silent = True)

param_grid={'booster' :['gbtree'],
                 'silent':[True],
                 'max_depth':[5,6,8],
                 'min_child_weight':[1,3,5],
                 'gamma':[0,1,2,3],
                 'nthread':[4],
                 'colsample_bytree':[0.5,0.8],
                 'colsample_bylevel':[0.9],
                 'n_estimators':[50],
                 'objective':['multi:softmax'],
                 'random_state':[2]}

grid_cv_XGB=GridSearchCV(model_XGB, param_grid=param_grid, cv=5 , n_jobs=-1)
grid_cv_XGB.fit(X_train, y_train)
print('최적 하이퍼 파라미터: ', grid_cv_XGB.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_XGB.best_score_))